<center><img src="https://i.imgur.com/fs4C0r1.png" width="700px"></center>

# Introduction

Hello everyone! Welcome to the <font color="#fa7703">"TReNDS Neuroimaging"</font> competition on Kaggle! In this competition, contestants are challenged to build machine learning models that can predict certain quantities related to <font color="#fa7703">normative age and brain connectivity based on fMRI scans of the patient's brain</font>. An accurate solution to this problem can open up new possibilities in the field of neuroimaging and neuroscience.

In this kernel, I will demonstrate how this problem can be <font color="#fa7703">broken down into a 2D image regression problem</font> and solved using pretrained ImageNet models like ResNet. I will use PyTorch v1.5 and Kaggle's NVIDIA Tesla P100 GPU to train the model.

<center><img src="https://i.imgur.com/Bev6ZXt.png" width="260px"></center>

# Acknowledgements

1. [<font color="#fa7703">PyTorch</font><font color="#5e5d5d"> ~ by PyTorch</font>](https://pytorch.org)
2. [<font color="#fa7703">Colorama</font><font color="#5e5d5d"> ~ by Jonathan Hartley</font>](https://github.com/tartley/colorama)
2. [<font color="#fa7703">Torchvision Models</font><font color="#5e5d5d"> ~ by PyTorch</font>](https://pytorch.org/docs/stable/torchvision/models.html)
3. [<font color="#fa7703">Torchviz</font><font color="#5e5d5d"> ~ by Sergey Zagoruyko</font>](https://github.com/szagoruyko/pytorchviz)
4. [<font color="#fa7703">Reading Matlab (.mat) Files and EDA</font><font color="#5e5d5d"> ~ by Manoj</font>](https://www.kaggle.com/mks2192/reading-matlab-mat-files-and-eda)
5. [<font color="#fa7703">Brain 3D plotly visualization</font><font color="#5e5d5d"> ~ by Vladislav Bakhteev</font>](https://www.kaggle.com/speedwagon/brain-3d-plotly-visualization)

# Contents

* [<font size=4 color="#fa7703">Preparing the ground</font>](#1)
    * [<font color="#5e5d5d">Import libraries</font>](#1.1)
    * [<font color="#5e5d5d">Set hyperparameters and paths</font>](#1.2)
    * [<font color="#5e5d5d">Load .csv data</font>](#1.3)
    * [<font color="#5e5d5d">Display few 2D slices along <i>x, y, and z</i> axes</font>](#1.4)

    
* [<font size=4 color="#fa7703">Modeling</font>](#2)
    * [<font color="#5e5d5d">Build PyTorch dataset</font>](#2.1)
    * [<font color="#5e5d5d">Build ResNet model</font>](#2.2)
    * [<font color="#5e5d5d">Visualize ResNet architecture</font>](#2.3)
    * [<font color="#5e5d5d">Define custom weighted absolute error loss</font>](#2.4)
    * [<font color="#5e5d5d">Define helper function for training logs</font>](#2.5)
    * [<font color="#5e5d5d">Split data into training and validation sets</font>](#2.6)
    * [<font color="#5e5d5d">Train model on GPU</font>](#2.7)


* [<font size=4 color="#fa7703">Takeaways</font>](#3)

# Preparing the ground <a id="1"></a>

## Import libraries <a id="1.1"></a> <font color="#fa7703" size=4>(for data loading, processing, and modeling on GPU)</font>

1. <font color="#fa7703">os</font> <font color="#5e5d5d"> ~ for managing paths</font>
2. <font color="#fa7703">gc</font> <font color="#5e5d5d"> ~ for garbage collection</font>
3. <font color="#fa7703">cv2</font> <font color="#5e5d5d"> ~ to process images</font>
4. <font color="#fa7703">h5py</font> <font color="#5e5d5d"> ~ to read 3D fMRI maps</font>
5. <font color="#fa7703">colored</font> <font color="#5e5d5d"> ~ to print colored text</font>
6. <font color="#fa7703">numpy</font> <font color="#5e5d5d"> ~ for linear algebra</font>
7. <font color="#fa7703">pandas</font> <font color="#5e5d5d"> ~ to manipulate tabular data</font>
8. <font color="#fa7703">random</font> <font color="#5e5d5d"> ~ to pick random slices</font>
9. <font color="#fa7703">tqdm</font> <font color="#5e5d5d"> ~ to generate progress bars</font>
10. <font color="#fa7703">matplotlib</font> <font color="#5e5d5d"> ~ to plot slices</font>
11. <font color="#fa7703">plotly</font> <font color="#5e5d5d"> ~ to plot 3D meshes</font>
13. <font color="#fa7703">torch</font> <font color="#5e5d5d"> ~ to build and train the neural network</font>
14. <font color="#fa7703">torchviz</font> <font color="#5e5d5d"> ~ to visualize PyTorch dynamic graphs</font>
15. <font color="#fa7703">torchvision</font> <font color="#5e5d5d"> ~ to download ResNet-18</font>

In [ ]:
!pip install -q colored
!pip install -q torchviz

In [ ]:
import os
import gc
import cv2
import time
import h5py
import colored
from colored import fg, bg, attr

from skimage import measure
from plotly.offline import iplot
from plotly import figure_factory as FF
from IPython.display import Markdown, display

import numpy as np
import pandas as pd
from random import randint
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
from torchviz import make_dot
torch.backends.cudnn.benchmark = True

import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.models import resnet18, densenet121, mobilenet_v2

## Set hyperparameters and paths <a id="1.2"></a> <font color="#fa7703" size=4>(adjust these to improve LB scores :D)</font>

1. Choose hyperparameters such as epochs, split percentage, learning rate, etc
2. Set important paths and directories to load data and train the model

In [ ]:
EPOCHS = 2
SPLIT = 0.8
LR = (1e-4, 1e-3)
MODEL_SAVE_PATH = "resnet_model"

W = 64
H = 64
BATCH_SIZE = 32
VAL_BATCH_SIZE = 32
DATA_PATH = '../input/trends-assessment-prediction/'

## Load .csv data <a id="1.3"></a> <font color="#fa7703" size=4>(to access 3D fMRI maps for training and validation)</font>

1. Load dataframes with IDs, targets, and tabular features
2. Extract relevant IDs for training and testing from the dataframes

In [ ]:
TEST_MAP_PATH = DATA_PATH + 'fMRI_test/'
TRAIN_MAP_PATH = DATA_PATH + 'fMRI_train/'

FEAT_PATH = DATA_PATH + 'fnc.csv'
TARG_PATH = DATA_PATH + 'train_scores.csv'
SAMPLE_SUB_PATH = DATA_PATH + 'sample_submission.csv'

TEST_IDS = [map_id[:-4] for map_id in sorted(os.listdir(TEST_MAP_PATH))]
TRAIN_IDS = [map_id[:-4] for map_id in sorted(os.listdir(TRAIN_MAP_PATH))]

In [ ]:
targets = pd.read_csv(TARG_PATH)
targets = targets.fillna(targets.mean())
sample_submission = pd.read_csv(SAMPLE_SUB_PATH)

features = pd.read_csv(FEAT_PATH)
test_df = features.query('Id in {}'.format(TEST_IDS)).reset_index(drop=True)
train_df = features.query('Id in {}'.format(TRAIN_IDS)).reset_index(drop=True)

In [ ]:
targets.head()

In [ ]:
features.head()

In [ ]:
sample_submission.head()

## Display few 2D slices along x, y, z <a id="1.4"></a> <font color="#fa7703" size=4>(to understand the slices we are dealing with)</font>

1. Calculate random slices of the fMRI map
2. Display the calculated slices using MatPlotLib

In [ ]:
def display_maps(idx):
    path = TRAIN_MAP_PATH + str(train_df.Id[idx])
    all_maps = h5py.File(path + '.mat', 'r')['SM_feature'][()]
    fig, ax = plt.subplots(nrows=3, ncols=5, figsize=(20, 12.5))

    plt.set_cmap('gray')
    for i in range(5):
        idx_1, idx_2, idx_3 = randint(0, 51), randint(0, 62), randint(0, 52)

        proj_1 = all_maps[:, idx_1, :, :].transpose(1, 2, 0)
        proj_2 = all_maps[:, :, idx_2, :].transpose(1, 2, 0)
        proj_3 = all_maps[:, :, :, idx_3].transpose(1, 2, 0)
        ax[0, i].imshow(cv2.resize(proj_1[:, :, 0], (H, W)))
        ax[1, i].imshow(cv2.resize(proj_2[:, :, 0], (H, W)))
        ax[2, i].imshow(cv2.resize(proj_3[:, :, 0], (H, W)))
        ax[0, i].set_title('Z-section {}'.format(i), fontsize=12)
        ax[1, i].set_title('Y-section {}'.format(i), fontsize=12)
        ax[2, i].set_title('X-section {}'.format(i), fontsize=12)

    plt.suptitle('Id: {}'.format(train_df.Id[idx])); plt.show()

### Map #0

Target values → {age: 57.44, domain1_var1: 30.57, domain1_var2: 62.55, domain2_var1: 55.33, domain2_var2: 51.43}

In [ ]:
display_maps(0)

### Map #1

Target values → {age: 59.58, domain1_var1: 50.97, domain1_var2: 67.47, domain2_var1: 60.65, domain2_var2: 58.31}

In [ ]:
display_maps(1)

### Map #2

Target values → {age: 71.41, domain1_var1: 53.15, domain1_var2: 58.01, domain2_var1: 52.42, domain2_var2: 62.54}

In [ ]:
display_maps(2)

# Modeling <a id="2"></a>

## Build PyTorch Dataset <a id="2.1"></a> <font color="#fa7703" size=4>(with map slicing and resizing)</font>

1. Retrieve all 53 fMRI maps for a given ID
2. Randomly slice each map along the *x, y,* and *z* axes to get 159 2D slices
3. Resize each 2D map to the shape (64, 64) using OpenCV-2 and concatenate all slices
4. Get targets for given ID and stack 159 times (each slice has the same target)
5. Return the calculated image tensors and target tensors for the given patient ID

In [ ]:
class TReNDSDataset(Dataset):
    def __init__(self, data, targets, map_path, is_train):
        self.data = data
        self.is_train = is_train
        self.map_path = map_path
        self.map_id = self.data.Id
        if is_train: self.targets = targets
            
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path = self.map_path + str(self.map_id[idx])
        all_maps = h5py.File(path + '.mat', 'r')['SM_feature'][()]
        cols = ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']
        
        idx_1, idx_2, idx_3 = randint(0, 51), randint(0, 62), randint(0, 52)
        proj_1 = cv2.resize(all_maps[:, idx_1, :, :].transpose(1, 2, 0), (H, W))
        proj_2 = cv2.resize(all_maps[:, :, idx_2, :].transpose(1, 2, 0), (H, W))
        proj_3 = cv2.resize(all_maps[:, :, :, idx_3].transpose(1, 2, 0), (H, W))
        features = np.concatenate([proj_1, proj_2, proj_3], axis=2).transpose(2, 0, 1)
        
        if not self.is_train:
            return torch.FloatTensor(features)
        else:
            i = self.map_id[idx]
            targets = self.targets.query('Id == {}'.format(i)).values
            targets = np.repeat(targets[:, 1:], 159, 0).reshape(-1, 5)
            return torch.FloatTensor(features), torch.FloatTensor(targets)

## Build ResNet model <a id="2.2"></a> <font color="#fa7703" size=4>(with a double dense head)</font>

1. Get ResNet-18 head (till AvgPool)
2. Add two Dense layers (16 and 5 neurons) on top
3. Reshape and tile image to match ResNet input dimensions
4. Pass reshaped image tensor through neural network and get output

In [ ]:
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        
        self.identity = lambda x: x
        self.dense_out = nn.Linear(16, 5)
        self.dense_in = nn.Linear(512, 16)
        self.resnet = resnet18(pretrained=True, progress=False)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])
        
    def forward(self, img):
        img = img.reshape(-1, 1, H, W)
        feat = self.resnet(img.repeat(1, 3, 1, 1))
        
        conc = self.dense_in(feat.squeeze())
        return self.identity(self.dense_out(conc))

## Visualize ResNet architecture<a id="2.3"></a> <font color="#fa7703" size=4>(with pytorchviz)</font>

In [ ]:
model = ResNetModel()
x = torch.randn(1, 3, 64, 64).requires_grad_(True)
y = model(x)
make_dot(y, params=dict(list(model.named_parameters()) + [('x', x)]))

In [ ]:
del model, x, y
gc.collect()

## Define custom weighted absolute error loss<a id="2.4"></a> <font color="#fa7703" size=4>(for backpropagation)</font>

1. Define weightage for each target
2. Calculate weighted absolute errors and take the average

In [ ]:
def weighted_nae(inp, targ):
    W = torch.FloatTensor([0.3, 0.175, 0.175, 0.175, 0.175])
    return torch.mean(torch.matmul(torch.abs(inp - targ), W.cuda()/torch.mean(targ, axis=0)))

## Define helper function for training logs <a id="2.5"></a> <font color="#fa7703" size=4>(to check training status)</font>

1. Retrieve training and validation metrics
2. Format metrics and display them during training

In [ ]:
def print_metric(data, batch, epoch, start, end, metric, typ):
    time = np.round(end - start, 1)
    time = "Time: %s{}%s s".format(time)

    if typ == "Train":
        pre = "BATCH %s" + str(batch-1) + "%s  "
    if typ == "Val":
        pre = "EPOCH %s" + str(epoch+1) + "%s  "
    
    fonts = (fg(216), attr('reset'))
    value = np.round(data.item(), 3)
    t = typ, metric, "%s", value, "%s"

    print(pre % fonts , end='')
    print("{} {}: {}{}{}".format(*t) % fonts + "  " + time % fonts)

## Split data into training and validation sets <a id="2.6"></a> <font color="#fa7703" size=4>(to validate performance properly)</font>

1. Split the data into training and validation sets
2. Define the test data loader to run inference after training

In [ ]:
val_out_shape = -1, 5
train_out_shape = -1, 5

split = int(SPLIT*len(train_df))
val = train_df[split:].reset_index(drop=True)
train = train_df[:split].reset_index(drop=True)

test_set = TReNDSDataset(test_df, None, TEST_MAP_PATH, False)
test_loader = DataLoader(test_set, batch_size=VAL_BATCH_SIZE)

## Train model on GPU <a id="2.7"></a> <font color="#fa7703" size=4>(NVIDIA Tesla P100)</font>

1. Define dataloders, model, optimizer, and learning rate scheduler
2. Train the model in batches and check validation performance at the end of each epoch
3. Save the model architecture and weights and generate testing predictions after training

In [ ]:
def train_resnet18():
    def cuda(tensor):
        return tensor.cuda()
   
    val_set = TReNDSDataset(val, targets, TRAIN_MAP_PATH, True)
    val_loader = DataLoader(val_set, batch_size=VAL_BATCH_SIZE)
    train_set = TReNDSDataset(train, targets, TRAIN_MAP_PATH, True)
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

    network = cuda(ResNetModel())
    optimizer = Adam([{'params': network.resnet.parameters(), 'lr': LR[0]},
                      {'params': network.dense_in.parameters(), 'lr': LR[1]},
                      {'params': network.dense_out.parameters(), 'lr': LR[1]}])

    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5,
                                  patience=2, verbose=True, eps=1e-6)
    start = time.time()
    for epoch in range(EPOCHS):
        batch = 1
        fonts = (fg(216), attr('reset'))
        print(("EPOCH %s" + str(epoch+1) + "%s") % fonts)

        for train_batch in train_loader:
            train_img, train_targs = train_batch
           
            network.train()
            network = cuda(network)
            train_preds = network.forward(cuda(train_img))
            train_targs = train_targs.reshape(train_out_shape)
            train_loss = weighted_nae(train_preds, cuda(train_targs))

            optimizer.zero_grad()
            train_loss.backward()

            optimizer.step()
            end = time.time()
            batch = batch + 1
            print_metric(train_loss, batch, epoch, start, end, metric="loss", typ="Train")
            
        print("\n")
           
        network.eval()
        for val_batch in val_loader:
            img, targ = val_batch
            val_preds, val_targs = [], []

            with torch.no_grad():
                img = cuda(img)
                network = cuda(network)
                pred = network.forward(img)
                val_preds.append(pred); val_targs.append(targ)

        val_preds = torch.cat(val_preds, axis=0)
        val_targs = torch.cat(val_targs, axis=0)
        val_targs = val_targs.reshape(val_out_shape)
        val_loss = weighted_nae(val_preds, cuda(val_targs))
        
        avg_preds = []
        avg_targs = []
        for idx in range(0, len(val_preds), 159):
            avg_preds.append(val_preds[idx:idx+159].mean(axis=0))
            avg_targs.append(val_targs[idx:idx+159].mean(axis=0))
            
        avg_preds = torch.stack(avg_preds, axis=0)
        avg_targs = torch.stack(avg_targs, axis=0)
        loss = weighted_nae(avg_preds, cuda(avg_targs))
        
        end = time.time()
        scheduler.step(val_loss)
        print_metric(loss, None, epoch, start, end, metric="loss", typ="Val")
        
        print("\n")
   
    network.eval()
    if os.path.exists(TEST_MAP_PATH):

        test_preds = []
        for test_img in test_loader:
            with torch.no_grad():
                network = cuda(network)
                test_img = cuda(test_img)
                test_preds.append(network.forward(test_img))
        
        avg_preds = []
        test_preds = torch.cat(test_preds, axis=0)
        for idx in range(0, len(test_preds), 159):
            avg_preds.append(test_preds[idx:idx+159].mean(axis=0))

        torch.save(network.state_dict(), MODEL_SAVE_PATH + ".pt")
        return torch.stack(avg_preds, axis=0).detach().cpu().numpy()

In [ ]:
print("STARTING TRAINING ...\n")

test_preds = train_resnet18()
    
print("ENDING TRAINING ...")

## Generate predictions <a id="2.8"></a> <font color="#fa7703" size=4>(to submit to the competition :D)</font>

In [ ]:
sample_submission.Predicted = test_preds.flatten()

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv', index=False)

# Takeaways <a id="3"></a>

1. <font color="#fa7703" size=3>The maps can be sliced and resized in order to convert this into an image regression problem.</font>
2. <font color="#fa7703" size=3>Incorporating the tabular data features can probably boost the performace of this model.</font>
3. <font color="#fa7703" size=3>Using bigger models with greater representational capacity (DenseNet, EfficientNet, etc) can boost scores.</font>